In [ ]:
#!pip install datasets
#!pip install transformers
#!pip install evaluate
from datasets import load_dataset

In [ ]:
food = load_dataset("food101", split="train[:5000]")
#split dataset
food = food.train_test_split(test_size=0.2)
#example
food["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=382x512>,
 'label': 10}

In [ ]:
labels = food["train"].features["label"].names

In [ ]:
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
id2label[str(79)]

'prime_rib'

In [ ]:
from transformers import AutoImageProcessor

In [ ]:
checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

In [ ]:
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [ ]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [ ]:
food = food.with_transform(transforms)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#!pip install transformers[torch]
training_args = TrainingArguments(
    output_dir="food_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,2.711800,2.525361,0.829000
2,1.639600,1.609706,0.896000


TrainOutput(global_step=186, training_loss=2.4524936983662267, metrics={'train_runtime': 527.1184, 'train_samples_per_second': 22.765, 'train_steps_per_second': 0.353, 'total_flos': 9.232831524962304e+17, 'train_loss': 2.4524936983662267, 'epoch': 2.98})

In [ ]:
trainer.save_model(output_dir="best_food")

In [ ]:
ds = load_dataset("food101", split="validation[:10]")
image = ds["image"][0]

In [ ]:
from transformers import pipeline
classifier = pipeline("image-classification", model="best_food")
classifier(image)

[{'score': 0.2840491533279419, 'label': 'beignets'},
 {'score': 0.019540566951036453, 'label': 'ramen'},
 {'score': 0.01433509774506092, 'label': 'bruschetta'},
 {'score': 0.01351894997060299, 'label': 'prime_rib'},
 {'score': 0.013120424933731556, 'label': 'hamburger'}]

In [ ]:
from transformers import AutoImageProcessor
import torch

image_processor = AutoImageProcessor.from_pretrained("best_food")
inputs = image_processor(image, return_tensors="pt")

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("best_food")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predicted_label = logits.argmax(-1).item()
model.config.id2label[predicted_label]

'beignets'